In [143]:
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import time
import random
from fake_useragent import UserAgent
#libs used to do web scraping and sending req

In [144]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.pakwheels.com/used-cars/honda/32"
}

#these are the headers to mimic th human behavior

In [71]:
base_url="https://www.pakwheels.com/used-cars/honda-lahore/201"#URL of the page

In [145]:
from dotenv import load_dotenv#importing
import os

In [146]:
load_dotenv()#load everything from env folder
mongo_url=os.getenv("DB_URL")
proxy_auth=os.getenv("proxy_key")

In [147]:
proxies={
    'http':f'http://{proxy_auth}',
    'https':f'http://{proxy_auth}'
}
#proxies list to save us from blocking 

In [148]:
session=requests.Session()#set session as making request for session 

In [76]:
client=MongoClient(mongo_url)
data_base=client["Honda_cars"]#set database name
collection=data_base["listings"]#set collection/table


In [ ]:
#this code is used to get and save all the pages of honda brand 
def get_all_pages_of_honda():
    
    for page in range(1, 90):  # first 90 pages
        url = f"{base_url}?page={page}&_pjax=%5Bdata-pjax-container%5D"
        print(f"Saving page {page}: {url}")

        r = requests.get(url, headers=headers)

        with open(f"pages/page_{page}.html", "w", encoding="utf-8") as f:
            f.write(r.text)

        time.sleep(1 + 2 * random.random())


In [ ]:
get_all_pages_of_honda()#calling function 

In [ ]:
import json
#importing json so that we can read and write json

In [ ]:
#from the saved html get all the links so that we can later scrap by using this link 
def extract_ads_link_and_title_from_html():
    urls=[]
    titles=[]
    for i in range (1,90):
        with open(f"pages/page_{i}.html","r",encoding="utf-8") as f:
            html=f.read()
        soup=BeautifulSoup(html,'html.parser')
        script_tag=soup.find("script", type="application/ld+json")
        data =json.loads(script_tag.string)
        listings=data.get("itemListElement", [])
        for j in listings:
            urls.append(j.get("url"))
            titles.append(j.get("title"))
    return urls, titles

In [ ]:
urls,titles=extract_ads_link_and_title_from_html()#calling function and store urls and title 

In [ ]:
print(len(urls)) #print to check its value

2613


In [ ]:
#this code is getting ada and save it so that later we can parse on it 
def get_ads_html():
    for i, url in enumerate( urls):
        print(f"saving add: {i}\n")
        r=session.get(url,headers=headers)
        with open (f"listings/listing_{i}.html","w",encoding="utf-8") as f: #open files and save as listing_{i} name 
            f.write(r.text)#open as written form
        time.sleep(1 + 2 * random.random())#mimic human behaviour add time delay 

In [ ]:
get_ads_html()# calling a function #save to db

saving add: 0

saving add: 1

saving add: 2

saving add: 3

saving add: 4

saving add: 5

saving add: 6

saving add: 7

saving add: 8

saving add: 9

saving add: 10

saving add: 11

saving add: 12

saving add: 13

saving add: 14

saving add: 15

saving add: 16

saving add: 17

saving add: 18

saving add: 19

saving add: 20

saving add: 21

saving add: 22

saving add: 23

saving add: 24

saving add: 25

saving add: 26

saving add: 27

saving add: 28

saving add: 29

saving add: 30

saving add: 31

saving add: 32

saving add: 33

saving add: 34

saving add: 35

saving add: 36

saving add: 37

saving add: 38

saving add: 39

saving add: 40

saving add: 41

saving add: 42

saving add: 43

saving add: 44

saving add: 45

saving add: 46

saving add: 47

saving add: 48

saving add: 49

saving add: 50

saving add: 51

saving add: 52

saving add: 53

saving add: 54

saving add: 55

saving add: 56

saving add: 57

saving add: 58

saving add: 59

saving add: 60

saving add: 61

saving add: 62

sa

In [ ]:
#This is the code that is extracting brands name and description from the saved htmls
def extract_brands_and_name():
    brands=[]
    names=[]
    descriptions=[]
    for i in range(len(urls)):#run loop until total ads
        try:
            with open(f"listings/listing_{i}.html","r",encoding="utf-8") as f:
                html=f.read()#opened in read format
            soup=BeautifulSoup(html,'html.parser')
            script_tags=soup.find_all("script", type="application/ld+json")# find the tag like this and store it so that we 
            #can extract the info from this tag as we want 
            for tag in script_tags:
                data=json.loads(tag.string)#loads json in data
                if isinstance(data, dict) and "Product" in data.get("@type", []):# if product exist 
                    brand=data.get("brand",{}).get("name")#get brand name
                    name=data.get("model")#get model name
                    description=data.get("description")#get description 
                    #basically it finds objects like these and tore their values and append to man var as I did below
                    brands.append(brand)
                    names.append(name)
                    descriptions.append(description)
                    break
        except:
            continue
    return names,brands,descriptions 

In [ ]:
names,brands,descriptions=extract_brands_and_name()#calling a function

In [ ]:
#this function get the url of the images so that we can get the image and evaluate on our model
def get_image_urls(total_files):
    all_ads_images_urls = []

    for i in range(total_files):
        with open(f"listings/listing_{i}.html", "r", encoding="utf-8") as f:
            html = f.read()

        soup = BeautifulSoup(html, "html.parser")
        gallery = soup.find("ul", class_="gallery")

        ad_images_url = []

        if gallery:
            for li in gallery.find_all("li"):#find the tag like this
                img = li.find("img")
                if img:
                    url = img.get("data-original")#if image found then go to the original line and get the line do not take thumb one 
                    if url:
                        ad_images_url.append(url)# create image array

        all_ads_images_urls.append(ad_images_url)# create array of arrays so that every ad contain more pics not one

    return all_ads_images_urls

In [ ]:
image_urls=get_image_urls(len(urls)) #calling a function

In [ ]:
#this function get the the inspection rating if available other wise store none
def get_inspection_report(total_files):
    ratings=[]
    for i in range(total_files):
        with open(f"listings/listing_{i}.html", "r", encoding="utf-8") as f:
            html = f.read()

        soup = BeautifulSoup(html, "html.parser")
        rating_div=soup.find("div",class_="right pull-right primary-lang")
        if not rating_div:
            ratings.append(None)
            continue 
        try:
            text = rating_div.text.strip()     # get ratings
            score = float(text.split("/")[0])  # 9.4 store only first one
            rating_int = int(round(score))     # 9 convert to int
            ratings.append(rating_int)
        except:
            ratings.append(None)
    return ratings

In [ ]:
ratings=get_inspection_report(len(urls))#calling a functions

In [ ]:
print(ratings)#check

[9, None, 7, None, 10, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 6, 10, 7, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 7, None, None, 6, None, None, None, None, None, None, None, None, None, 6, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 8, None, None, None, 9, None, None, None, None, None, None, None, None, None, None, None, 7, None, None, None, None, None, None, 6, None, None, None, None, None, 10, None, None, None, None, 6, None, 9, 10, None, None, None, None, None, 8, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 9, None, 9, 7, None, None, None, None, None, None, None, None, None, 9, None, None, None, None,

In [ ]:
print (image_urls[0])#check

['https://cache4.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462071.webp', 'https://cache3.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462072.webp', 'https://cache2.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462076.webp', 'https://cache2.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462073.webp', 'https://cache3.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462075.webp', 'https://cache3.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462074.webp', 'https://cache1.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462081.webp', 'https://cache1.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462078.webp', 'https://cache4.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462077.webp', 'https://cache3.pakwheels.com/ad_pictures/1364/honda-city-1-5l-aspire-cvt-2022-136462079.webp', 'https://cache4.pakwheels.com/ad_pictur

In [ ]:
#title=[....]
#urls=[....]
# brands = [...]
# names = [...]
# descriptions = [...]
# images = [...]      # list of lists
# ratings = [...]
#variables name 

In [ ]:
#this function is basically making array of array of listing and each index contain one ad so that we cn push it into DB
def save_to_mongodb(total_files):
    listings=[]
    for i in range(total_files):
        listing={
        "URL":urls[i],
        "Title":titles[i],
        "brand": brands[i],
        "name": names[i],
        "description": descriptions[i],
        "images": image_urls[i],
        "rating": ratings[i]
        }
        listings.append(listing)
    return listings

In [ ]:
listings=save_to_mongodb(len(urls))#calling a function

In [152]:
#save to db
collection.insert_many(listings)

InsertManyResult([ObjectId('69666b240bd2b688bfa9597c'), ObjectId('69666b240bd2b688bfa9597d'), ObjectId('69666b240bd2b688bfa9597e'), ObjectId('69666b240bd2b688bfa9597f'), ObjectId('69666b240bd2b688bfa95980'), ObjectId('69666b240bd2b688bfa95981'), ObjectId('69666b240bd2b688bfa95982'), ObjectId('69666b240bd2b688bfa95983'), ObjectId('69666b240bd2b688bfa95984'), ObjectId('69666b240bd2b688bfa95985'), ObjectId('69666b240bd2b688bfa95986'), ObjectId('69666b240bd2b688bfa95987'), ObjectId('69666b240bd2b688bfa95988'), ObjectId('69666b240bd2b688bfa95989'), ObjectId('69666b240bd2b688bfa9598a'), ObjectId('69666b240bd2b688bfa9598b'), ObjectId('69666b240bd2b688bfa9598c'), ObjectId('69666b240bd2b688bfa9598d'), ObjectId('69666b240bd2b688bfa9598e'), ObjectId('69666b240bd2b688bfa9598f'), ObjectId('69666b240bd2b688bfa95990'), ObjectId('69666b240bd2b688bfa95991'), ObjectId('69666b240bd2b688bfa95992'), ObjectId('69666b240bd2b688bfa95993'), ObjectId('69666b240bd2b688bfa95994'), ObjectId('69666b240bd2b688bfa959